In [4]:
import numpy as np
import pandas as pd
import tensorflow as tf

import os

from helpers_datacup import *
from tf_helpers_datacup import *

In [5]:
performance  = pd.read_csv('train/performance_train.csv')
facturation  = pd.read_csv('train/facturation_train.csv')
payments     = pd.read_csv('train/paiements_train.csv')
transactions = pd.read_csv('train/transactions_train.csv')

customer_ids = performance['ID_CPTE']

print('Proportion of clients who default:', sum(performance['Default'])/len(performance))

Proportion of clients who default: 0.19336134453781512


In [26]:
class customer:
    
    def __init__(self, customer_id, performance, facturation, payments, transactions):
        
        self.customer_id  = customer_id
        self.performance  = performance
        self.facturation  = facturation.sort_values(by='PERIODID_MY')
        self.payments     = payments.sort_values(by='TRANSACTION_DTTM')
        self.transactions = transactions.sort_values(by=['TRANSACTION_DTTM'])
        
        self.assessment = list(performance['PERIODID_MY'])[0]
        self.default    = list(performance['Default'])[0]
        
        self.total_balance_change = facturation['CurrentTotalBalance'].values[1:] - facturation['CurrentTotalBalance'].values[:-1]
        self.cash_balance
        
        
def generate_clients(customer_ids, *dfs):

    for cus in customer_ids:
        yield [cus] + [df[df['ID_CPTE'] == cus] for df in dfs]

'''
clients is a list of client objects, with their information collected
into instances
'''
        
client_generator = generate_clients(customer_ids, performance, facturation, payments, transactions)
#clients = [customer(*client_info) for client_info in client_generator]
test_client = customer(*next(client_generator))

In [28]:
test_client.facturation

,ID_CPTE,PERIODID_MY,StatementDate,CurrentTotalBalance,CashBalance,CreditLimit,DelqCycle
1,99690111,2014-11-01,2014-11-03,866.00,0.00,12000.0,0
7,99690111,2014-12-01,2014-12-03,1151.85,0.00,12000.0,0
10,99690111,2015-01-01,2015-01-02,2298.96,350.00,12000.0,0
8,99690111,2015-02-01,2015-01-31,4045.67,1148.45,16200.0,0
11,99690111,2015-03-01,2015-03-03,5926.20,2567.25,16200.0,0
12,99690111,2015-04-01,2015-03-31,6916.62,3307.33,16200.0,0
0,99690111,2015-05-01,2015-05-03,8497.84,4293.12,16200.0,0
2,99690111,2015-06-01,2015-05-31,10790.95,5224.44,16200.0,0
13,99690111,2015-07-01,2015-07-05,10560.00,5127.54,16200.0,0
5,99690111,2015-08-01,2015-08-02,10610.05,4753.35,16200.0,0


In [10]:
def predictor(x_train, y_train, x_test, batch_size):
    '''
    1 convolution layer + LSTM + 2 fc layers
    '''
    
    EPOCHS = 3
    
    _, num_features  = x_train[0].shape
    num_hidden_cells = 8
    
    
    tf.reset_default_graph()
    
    x = tf.placeholder(dtype=tf.float32, shape=[None, ])
    y = tf.placeholder(dtype=tf.float32, shape=[None, 1])
    
    conv_1 = conv_pool_1d_layer(_input=x,
                                filter_size=3, 
                                num_input_channels=num_features,
                                num_output=8,
                                prefix='conv_1')
    
    print(conv_1.get_shape().as_list())
    
    lstm_cell  = tf.nn.rnn_cell.MultiRNNCell([tf.nn.rnn_cell.BasicLSTMCell(num_units=num_hidden_cells)])
    rnn_output = tf.nn.dynamic_rnn(lstm_cell, inputs=conv_1)

In [16]:
feature_1 = facturation['CurrentTotalBalance'].subtract(facturation['CashBalance'])
feature_2 = facturation['CurrentTotalBalance']

In [15]:
facturation['CurrentTotalBalance'].subtract(facturation['CashBalance'])

0         4204.72
1          866.00
2         5566.51
3         7602.38
4         7928.02
5         5856.70
6         6800.66
7         1151.85
8         2897.22
9         8328.16
10        1948.96
11        3358.95
12        3609.29
13        5432.46
14         243.41
15         489.72
16         260.00
17         266.06
18         416.00
19         210.08
20          -2.50
21         809.12
22         669.52
23         410.88
24        1057.83
25         234.00
26         392.20
27         219.00
28         135.00
29         229.84
           ...   
166513    2945.71
166514    1114.86
166515    2267.34
166516    1733.49
166517    1944.04
166518    4091.16
166519    2995.56
166520    2387.64
166521    2424.24
166522    7943.25
166523    2135.12
166524    2221.00
166525    2218.11
166526    1897.11
166527    1789.00
166528    2016.00
166529    2140.34
166530    2727.00
166531    2500.02
166532    2036.26
166533    1609.00
166534    2023.37
166535    1955.20
166536    2276.88
166537    